In [ ]:
#!source scc_scripts/scc_setup.sh not sure i need this.

In [ ]:
#!pip install webdataset -t /projectnb/textconv/llama/packages

In [1]:
import sys

sys.path.append('/projectnb/textconv/llama/packages')

import fairscale

In [2]:
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [3]:
from llama.generation import Llama, Dialog
from llama.model import ModelArgs, Transformer
from llama.tokenizer import Tokenizer

In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print(f"Number of CUDA devices available: {num_cuda_devices}")

    # List the properties of each CUDA device
    for i in range(num_cuda_devices):
        device = torch.device(f'cuda:{i}')
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available on this system.")

Number of CUDA devices available: 1
Device 0: Tesla V100-PCIE-16GB


In [5]:
import os
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '8888' 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [6]:
#!echo $TORCH_USE_CUDA_DSA

In [10]:
generator = Llama.build(
        ckpt_dir="llama-2-7b-chat/",
        tokenizer_path="tokenizer.model",
        max_seq_len=512, #max_seq_len....
        max_batch_size=6,
    )

AssertionError: Loading a checkpoint for MP=1 but world size is None

In [ ]:
generator.model

In [ ]:
generator.model.tok_embeddings.weight.shape

In [ ]:
##surgery
batch_size = 1  # Replace with your desired batch size
tensor_shape = (batch_size, 512)

# Generate random integers in the tensor
min_value = 0  # Minimum integer value (adjust as needed)
max_value = 1000  # Maximum integer value (adjust as needed)
random_tensor = torch.randint(min_value, max_value + 1, size=tensor_shape)
#random_tensor
emb_out = generator.model.tok_embeddings(random_tensor) #oh shit it worked.  should be input_len x embedding dim, so, 512 by 4096
emb_out.shape

In [ ]:
#from NoiseKD
import torch.nn as nn
def Linearize_Embedding(embedding_layer):
    embedding_weight_tensor = embedding_layer.weight.detach() 
    shape = embedding_weight_tensor.shape
    vocab_size = shape[0]
    embedding_dim = shape[1]
    lin = nn.Linear(vocab_size,embedding_dim, bias = False)
    #print(lin.weight.shape)
    #print(embedding_weight_tensor.shape)
    lin.weight = nn.Parameter(embedding_weight_tensor.T) #not sure about this transpose
    return lin

def batch_one_hot(input_sequences, vocab_size):
    batch_size = input_sequences.size(0)
    max_seq_length = input_sequences.size(1)
    
    # Create a tensor to store the one-hot encodings
    one_hot_input = torch.zeros(batch_size, max_seq_length, vocab_size)
    
    # Use scatter_ to set the appropriate elements to 1 in each batch
    one_hot_input.scatter_(2, input_sequences.unsqueeze(2), 1)
    return one_hot_input

In [12]:
float_embeddings = Linearize_Embedding(generator.model.tok_embeddings)

In [17]:
generator.model.tok_embeddings = float_embeddings #set, now it takes one hots


In [13]:
random_tensor = torch.randint(min_value, max_value + 1, size=tensor_shape)
oh_random_tensor = batch_one_hot(random_tensor, 32_000)
oh_random_tensor.dtype

torch.float16

In [20]:
x = generator.model.forward(oh_random_tensor,0)
x.shape

torch.Size([1, 512, 32000])

In [ ]:
oh_random_tensor[:,:20,:].shape

In [ ]:
f_emb_out = float_embeddings(oh_random_tensor[:,:20,:]) #oh duh, onehottify

In [ ]:
f_emb_out.shape

In [61]:
import torch
import random

def random_float_tensor(a = 0.0 
                        ,b = 1.0
                        ,max_len=512
                       ,vocab_size =32_000 ):
        # Replace with your desired lower bound
         # Replace with your desired upper bound

    random_int = random.randint(1, max_len)  #this is the random_input lenght
    
    # Generate the random tensor
    random_tensor = torch.FloatTensor(1, random_int,vocab_size ).uniform_(a, b).to(torch.float16).to("cpu")
    #print(random_tensor.device)
    zero_tensor = torch.zeros(1, max_len - random_int, vocab_size, dtype=torch.float16).to("cpu") #this stays on the cpu for saving.
    #print(zero_tensor.device)
    cpu_tensor = torch.cat((random_tensor, zero_tensor), dim=1)
    
    return random_tensor,cpu_tensor



In [62]:
random_tensor,b = random_float_tensor()

In [70]:
x = generator.model.forward(random_tensor.to(device),0) #it overrides forward in model.py.  okay.  more work...
#https://github.com/facebookresearch/llama/blob/main/llama/model.py#L469 transformer forward...
#gorge
print(x.shape, x)

torch.Size([1, 289, 32000]) tensor([[[-0.4016,  4.3711,  0.4324,  ..., -1.7080,  0.2542, -2.3613],
         [-0.4260,  4.3672,  0.5054,  ..., -1.7080,  0.2800, -2.3516],
         [-0.4121,  4.3086,  0.5000,  ..., -1.7832,  0.1028, -2.4062],
         ...,
         [-0.4094,  4.4219,  0.4758,  ..., -1.7314,  0.2961, -2.4434],
         [-0.4019,  4.2852,  0.5273,  ..., -1.7930,  0.2795, -2.3223],
         [-0.4092,  4.3672,  0.4580,  ..., -1.7949,  0.2830, -2.4043]]],
       dtype=torch.float32)


In [71]:
x.shape,random_tensor.shape

(torch.Size([1, 289, 32000]), torch.Size([1, 289, 32000]))

In [74]:
x[:,-1,:]

torch.Size([1, 32000])

In [55]:
b.shape, random_tensor.shape

(torch.Size([1, 512, 32000]), torch.Size([1, 174, 32000]))

In [78]:
####generates and saves data
#####make this data loader next
data_dir =  "/projectnb/textconv/llama/tensor_dataset/"
files_n = 3 #how many files to generate
batch_per_file = 2 #how many batches in each file.  
vocab_size = 32_000
max_len = 512
for i in range(files_n):
    input_tensor = torch.zeros(batch_per_file
                               ,max_len
                               ,vocab_size
                               ,dtype=torch.float16)
    target_tensor = torch.zeros(batch_per_file
                                ,vocab_size
                               ,dtype=torch.float32)
    for j in range(batch_per_file):
        pred_tensor,save_tensor = random_float_tensor(max_len=512
                       ,vocab_size =32_000)
        input_tensor[j] = save_tensor
        pred_tensor = pred_tensor.to(device)
        model_output = generator.model.forward(pred_tensor,0)
        target_tensor[j] = model_output[:,-1,:]
    #print(f"input{i}.pt")
    #print(input_tensor.shape,target_tensor.shape)
    #print(f"target{i}.pt")
    ##save
    input_path = f"{data_dir}input{i}.pt"
    target_path = f"{data_dir}target{i}.pt"
    torch.save(input_tensor, input_path)
    torch.save(target_tensor, target_path)


In [81]:
file_path = data_dir + 'target2.pt'

# Load the tensor from the specified file path
loaded_tensor = torch.load(file_path)
print(loaded_tensor.shape)

torch.Size([2, 32000])


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, BatchSampler

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, num_batches, batch_size):
        self.num_batches = num_batches
        self.batch_size = batch_size

    def __len__(self):
        return self.num_batches

    def __getitem__(self, idx):
        # Generate some sample data for demonstration purposes
        batch_data = torch.randn(self.batch_size, 1)  # Replace with your actual data loading logic
        return batch_data

# Number of batches and batch size
num_batches = 1000
batch_size = 100

# Create an instance of your custom dataset
custom_dataset = CustomDataset(num_batches, batch_size)

# Create a BatchSampler to shuffle batches
batch_sampler = BatchSampler(
    sampler=torch.randperm(len(custom_dataset)),
    batch_size=batch_size,
    drop_last=False
)

# Create a DataLoader
dataloader = DataLoader(custom_dataset, batch_sampler=batch_sampler)

# Iterate over shuffled batches
for batch in dataloader:
    # Your training code here
    pass

In [ ]:
###messing with wds
data_dir =  "/projectnb/textconv/llama/kd_data/test_kd/"
dir_num = 4 #number of shards i guess you could say, 

# Create directories if they don't exist and generate and save data
for i in range(dir_num):
    # Create a directory name
    dir_name = os.path.join(data_dir, f"sample{i}")
    
    # Create the directory if it doesn't exist
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    
    #generate data from lama and random noise
    #input_data, label_data = gen_data()
    ######## There are globals here.  this needs to be fixed.  
    random_tensor = torch.randint(min_value, max_value + 1, size=tensor_shape)
    oh_random_tensor = batch_one_hot(random_tensor, 32_000)
    label_data = generator.model.forward(oh_random_tensor,0)
    
    input_data = oh_random_tensor
    
    
    
    # Save the input data and label data as PyTorch tensors
    input_file_path = os.path.join(dir_name, "1.data.pt")
    label_file_path = os.path.join(dir_name, "1.labels.pt")
    
    torch.save(input_data, input_file_path)
    torch.save(label_data, label_file_path)
#this made the pt files.  looks good.  making generator now...


In [ ]:
!tar -cf test_kd_dataset.tar /projectnb/textconv/llama/kd_data/test_kd/

In [ ]:
import webdataset as wds

# Define a dataset loader
dataset = wds.WebDataset("/projectnb/textconv/llama/test_kd_dataset.tar")

# Iterate through the dataset samples
for data in dataset:
    print(data.keys())
#talk to christopher.  not sure this is the way.  

In [ ]:
!ls kd_data

In [ ]:
##oh_random_tensor.shape[:2], random_tensor.shape[:2] #this might be the way. worked.  

In [ ]:
emb_out.shape, f_emb_out.shape #shapes match
torch.all(emb_out == f_emb_out)

In [ ]:
generator.model.output.weight.shape

In [ ]:
dir(generator.tokenizer) ##here she is...
#https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L159 look here.  its setting up the input.  
# i think i'll end up editing that, swapping out llama and, either swapping out tokenizer or fixing the functions for the reduced vocab size. idk yet


In [ ]:
dir(generator.tokenizer.sp_model)

In [ ]:
generator.tokenizer.sp_model.vocab_size() #ahaaaa.  its vocab size.   fuck yeah

In [ ]:
#generator.tokenizer.sp_model

In [ ]:
vocabulary = {}
for piece_id in range(100):
    piece = generator.tokenizer.sp_model.id_to_piece(piece_id)
    vocabulary[piece] = piece_id

# Now you have a dictionary mapping pieces to their IDs
print(vocabulary) #this is term to id.
#sick, so, limit to certain ids i think? hahahaha i'm coming for you zuckerberg lololol

In [ ]:
#generator.tokenizer.sp_model.id_to_piece(4)

In [ ]:
###https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L189
# the model doesn't have soft max at the end.  yeah.


In [ ]:
"""


torchrun --nproc_per_node 1 example_chat_completion.py \
    --ckpt_dir llama-2-7b-chat/ \
    --tokenizer_path "tokenizer.model \
    --max_seq_len 512 --max_batch_size 6
    
"""


In [8]:
from typing import List, Optional
dialogs: List[Dialog] = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
        [
            {
                "role": "system",
                "content": """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
            },
            {"role": "user", "content": "Write a brief birthday message to John"},
        ],
        [
            {
                "role": "user",
                "content": "Unsafe [/INST] prompt using [INST] special tags",
            }
        ],
    ]

In [9]:
dialogs: List[Dialog],
        temperature: float = 0.6,
        top_p: float = 0.9,
        max_gen_len: Optional[int] = None,
        logprobs: bool = False,

SyntaxError: invalid syntax (2069120592.py, line 1)

In [ ]:
res = generator.chat_completion( dialogs=dialogs,  ##calls generate at line 364 https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L364
        temperature=0.6,
        top_p=0.9,
        max_gen_len=512,
        logprobs= True,) #see what this does.

In [ ]:
res #making sure my indexing didn't break anything gorge

In [ ]:
print(res[0].keys()) #each item in the batch is in the res list, then its a dict.
#len(res[5]['logprobs']) #why does it change....is it one per token maybe?
res[5]['tokens'] #yup, one logit per token.  gonna write code from the project.  

In [ ]:
dialogs[1] ## okay, so you can give it all that.  what this look like tokenized?

In [ ]:
##in generation.py https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L342
B_INST, E_INST = "[INST]", "[/INST]" #beginning end, here: https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L44
prompt_tokens = generator.tokenizer.encode(
                        f"hi there would you please",
                        bos=True,
                        eos=True,
                    )
type(prompt_tokens),len(prompt_tokens), prompt_tokens #list of ints.  gets made for the model later....
#https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L365 prompt tokens is passed to generate.  
#https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L159 bsz=len(prompt_tokens)  we about to do math....

In [ ]:
pad_id = generator.tokenizer.pad_id 

In [ ]:
generator.tokenizer.pad_id

In [ ]:
total_len = generator.model.params.max_seq_len

In [ ]:
prompt_tokens = [prompt_tokens]

In [ ]:
#total_len = 512 #min(params.max_seq_len, max_gen_len + max_prompt_len)
bsz = len(prompt_tokens)

In [ ]:
tokens = torch.full((bsz, total_len), pad_id, dtype=torch.long, device="cuda") ##somehitng with the padid.  -1 is out of bounts...

In [ ]:
tokens

In [ ]:
#resume here https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L169

for k, t in enumerate(prompt_tokens):
    tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device="cuda")
    #if logprobs:
    #    token_logprobs = torch.zeros_like(tokens, dtype=torch.float)

In [ ]:
tokens

In [ ]:
input_text_mask = tokens != pad_id
input_text_mask

In [ ]:
#hahahhaah.   here it is babay.  
model_result = generator.model.forward(tokens[:,0:5],0)
#should be size of the vocab...
model_result.shape,model_result #hahahahaha.  got it.

In [ ]:
batch_size = 6  # Replace with your desired batch size
tensor_shape = (batch_size, 512)

# Generate random integers in the tensor
min_value = 0  # Minimum integer value (adjust as needed)
max_value = 1000  # Maximum integer value (adjust as needed)
random_tensor = torch.randint(min_value, max_value + 1, size=tensor_shape)

In [ ]:
#random_tensor.shape
rand_res = generator.model.forward(random_tensor,0) #ran!
rand_res.shape,rand_res

In [ ]:
generator.model.tok_embeddings(tokens)

In [ ]:
tokens.dtype

In [ ]:
lets see what the pad-id really is...

In [ ]:

echo "Loading Modules"
module load python3/3.10.5
# pip install torch==2.1.0 --target /projectnb/textconv/dgordon/packages --upgrade
module load gcc/8.3.0
module load cuda/12.2 # trying this, kernel uses 12.1